This notebook contains the code needed to generate the data which tracks the POIS and diversity througout runs of SHADE and L-SHADE.
It is based on the pyade project by David Criado Ramón (https://github.com/xKuZz/pyade), with some some modifications (class-based structure, SDIS, initialization). 
This code is part of the paper "The Importance of Being Restrained" DETAILS + CITATION INFO TO ADD HERE

In [ ]:
#Basic setup part: importing needed libraries + pyade classes defining the DE-wrapper
import numpy as np
import ioh # Used version 0.3.2.4
import pickle as pkl
from pyade.shade import shade
from pyade.ilshade import ilshade
from pyade.lshade import lshade
from pyade.mpede import mpede
from pyade.sade import sade
from pyade.de import de

versions = ["shade", "ilshade", "lshade", "mpede", "sade", "de"]
sdiss = ['saturate', 'mirror', 'COTN', 'unif_resample', 'toroidal']

class DE_evaluator():
    def __init__(self, version, corr_method):
        self.alg = eval(f"{version}")
        self.corr_method = corr_method
    
    def __call__(self, func, seed = None):
        self.p = self.alg.get_default_params(dim=func.meta_data.n_variables)
        self.p['bounds'] = np.array([[-5, 5]] * func.meta_data.n_variables)
        self.p['func'] = func
        self.p['corr_method'] = self.corr_method
        self.p['seed'] = seed
        self.alg.apply(**self.p)

In [ ]:
%%capture cap --no-stderr

#Set parameters for selecting what version + functions to use. 
#Note!!!: Works only when the flag '_write_output' is set to True in pyade/commons.py

de_version = 'lshade' # We use both shade and lshade for the paper
dim = 30 # We use dims 5 and 30 for the paper
iid = 1 # we use only iid 1 in the paper to save some computation time

print("/")
for fid in range(1,25):
    for sdis in sdiss:
        for rep in range(5):
            de_e = DE_evaluator(de_version, sdis)
            func = ioh.get_problem(fid,iid,dim)
            de_e(func, rep)
            print("/")

In [ ]:
#Write raw log to file
with open(f'log_all_fcts_sdis_5rep_iid{iid}_{de_version}_{dim}d.txt', 'w') as f:
    f.write(cap.stdout)